In [117]:
import warnings
warnings.filterwarnings('ignore')

In [118]:
from numpy.random import seed
seed(42)

import tensorflow
tensorflow.random.set_seed(42)

In [119]:
import os
import pandas as pd
import math
import numpy as np

Get sorted cases id

In [120]:
import wget
import os.path

if(not os.path.exists('finale.csv')):
    wget.download('https://data.4tu.nl/ndownloader/files/23993303')
    
df_finale = pd.read_csv('finale.csv', index_col=0).reset_index()

df_finale['Complete Timestamp'] = pd.to_datetime(df_finale['Complete Timestamp'])
sorted_time_cases = df_finale.sort_values('Complete Timestamp')['Case ID'].drop_duplicates().values

In [121]:
df_final = pd.read_csv('finale_time_features.csv')

In [122]:
df_final.shape

(21348, 23)

In [123]:
df_final.columns

Index(['Unnamed: 0', 'Case ID', 'Activity', 'Resource', 'Complete Timestamp',
       'Variant', 'Variant index', 'Variant.1', 'seriousness', 'customer',
       'product', 'responsible_section', 'seriousness_2', 'service_level',
       'service_type', 'support_section', 'workgroup',
       'Complete Timestamp Shift', 'Duration', 'Duration Float',
       'Time to conclusion', 'Passed Time', 'Step'],
      dtype='object')

In [124]:
just_closed = False
mode = 'bukhsh_partition'
#mode = 'silva_partition'

In [125]:
if(just_closed):
    closed_cases = []

    for name, group in df_final.groupby('Case ID'):
        last_step = group.tail(1)['Activity'].values[0]
        if(last_step == 'Closed'):
            closed_cases.append(group['Case ID'].values[0])

    df_final = df_final[df_final['Case ID'].isin(closed_cases)]     

In [126]:
df_final['Activity Cat'] = df_final['Activity'].astype('category').cat.codes

In [127]:
df_final[['Activity Cat', 'Step', 'Duration Float', 'Passed Time', 'Time to conclusion']].head(9)

,Activity Cat,Step,Duration Float,Passed Time,Time to conclusion
0,0,1,0.0000,0.0000,31.0087
1,11,2,16.0084,0.0000,15.0003
2,9,3,0.0001,15.0002,15.0002
3,1,4,15.0002,15.0003,0.0000
4,1,5,0.0000,31.0087,0.0000
5,0,1,0.0000,0.0000,30.9822
6,11,2,5.8750,25.1053,25.1072
7,9,3,0.0019,25.1072,25.1053
8,1,4,25.1053,30.9822,0.0000


In [128]:
one_hot = pd.get_dummies(df_final['Activity Cat'])

# Join the encoded df
df_final = df_final.join(one_hot)

In [129]:
from sklearn.model_selection import train_test_split

print(mode)
if (mode == 'bukhsh_partition'):    
    train_cases = pd.read_csv('bukhsh_data/bukhsh_remaining_time_train.csv')['case_id'].unique()
    test_cases = pd.read_csv('bukhsh_data/bukhsh_remaining_time_test.csv')['case_id'].unique()
    val_cases, test_cases  = train_test_split(test_cases, test_size=0.5, 
                                            shuffle=True,
                                            random_state=42
                                           )
else:
    train_cases, test_cases  = train_test_split(sorted_time_cases, test_size=0.4, 
                                            shuffle=False, 
                                            random_state=42
                                           )
    
    val_cases, test_cases  = train_test_split(test_cases, test_size=0.5, 
                                            shuffle=True,
                                            random_state=42
                                           )

bukhsh_partition


In [130]:
len(train_cases), len(test_cases) , len(val_cases)

(3664, 458, 458)

In [131]:
print(
    df_finale[df_finale['Case ID'].isin(train_cases)]['Complete Timestamp'].min(),
    df_finale[df_finale['Case ID'].isin(train_cases)]['Complete Timestamp'].max()
)

2010-01-13 08:40:25 2014-01-03 13:20:58


In [132]:
print(
    df_finale[df_finale['Case ID'].isin(val_cases)]['Complete Timestamp'].min(),
    df_finale[df_finale['Case ID'].isin(val_cases)]['Complete Timestamp'].max()
)

2010-01-13 12:30:37 2014-01-02 08:32:07


In [133]:
print(
    df_finale[df_finale['Case ID'].isin(test_cases)]['Complete Timestamp'].min(),
    df_finale[df_finale['Case ID'].isin(test_cases)]['Complete Timestamp'].max()
)

2010-02-04 08:20:45 2014-01-03 10:08:18


Overriding cases checking

In [134]:
print(
    set(test_cases).intersection(set(train_cases)),
    set(test_cases).intersection(set(val_cases)),
    set(train_cases).intersection(set(val_cases))
)

set() set() set()


Dataframes creation

In [135]:
df_train = df_final[df_final['Case ID'].isin(train_cases)]
df_test = df_final[df_final['Case ID'].isin(test_cases)]
df_val = df_final[df_final['Case ID'].isin(val_cases)]

Features selection

In [136]:
X_train = df_train[['Step', 'Duration Float', 'Passed Time', 0,1,2,3,4,5,6,7,8,9,10,11,12,13]]
X_train = X_train.values

y_train = df_train[['Time to conclusion']].values


X_test = df_test[['Step', 'Duration Float', 'Passed Time', 0,1,2,3,4,5,6,7,8,9,10,11,12,13]]
X_test = X_test.values

y_test = df_test[['Time to conclusion']].values


X_val = df_val[['Step', 'Duration Float', 'Passed Time', 0,1,2,3,4,5,6,7,8,9,10,11,12,13]]
X_val = X_val.values

y_val = df_val[['Time to conclusion']].values

In [137]:
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

regressors = [  
        {'nome' : 'DummyRegressor', 'regressor':  DummyRegressor(strategy='mean')},
        {'nome': 'LinearRegression', 'regressor': LinearRegression(fit_intercept=False, normalize=True)},
        {'nome': 'RandomForestRegressor',  'regressor': RandomForestRegressor(n_estimators=20, criterion='mse', max_features='auto', 
                                                                              bootstrap=True, warm_start=True, max_depth=19, random_state=0)},
]

In [138]:
import time
from sklearn.metrics import mean_absolute_error

cols = [
        'Nome do modelo', 'MAE', 
        'MAE (val)'
       ]

data_result = []

for regressor in regressors:    
    t1 = time.time()
    regr = regressor['regressor'].fit(X_train, y_train)
    tf = time.time()
    
    y_pred = regr.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    
    y_val_pred = regr.predict(X_val)
    mae_val = mean_absolute_error(y_val, y_val_pred)    
        
    line = [
        regressor['nome'],
        mae,
        mae_val
    ]
    data_result.append(line)

pd.DataFrame(data_result, columns=cols).sort_values(by='MAE', ascending=True)

,Nome do modelo,MAE,MAE (val)
2,RandomForestRegressor,3.476310,3.406881
1,LinearRegression,5.828287,5.929979
0,DummyRegressor,14.825708,14.998173


Deep learning model definition

In [139]:
import keras
from keras_self_attention import SeqSelfAttention

In [140]:
from tensorflow import keras

from keras.layers import Dropout
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Bidirectional
from keras.layers import PReLU
from keras.layers import Embedding


from keras.utils.vis_utils import plot_model

from keras.callbacks import EarlyStopping
from tqdm.keras import TqdmCallback

In [141]:
from keras.layers.core import Activation
from keras.initializers import Constant

from keras.regularizers import l2

In [142]:
from keras_multi_head import MultiHead
from keras_multi_head import MultiHeadAttention

model = keras.models.Sequential(name='')

model.add(Dense(X_train.shape[1], 
                input_dim=X_train.shape[1], 
                activation='relu', 
                name='Densa'
                ))

model.add(Dropout(0.05, name='Regularizacao'))

model.add(keras.layers.Flatten(name='Nivelamento'))
model.add(Dense(1, activation='relu', name='Saida'))

model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Densa (Dense)               (None, 17)                306       
                                                                 
 Regularizacao (Dropout)     (None, 17)                0         
                                                                 
 Nivelamento (Flatten)       (None, 17)                0         
                                                                 
 Saida (Dense)               (None, 1)                 18        
                                                                 
Total params: 324
Trainable params: 324
Non-trainable params: 0
_________________________________________________________________


In [143]:
nadam = keras.optimizers.Nadam(
    learning_rate=0.01, beta_1=0.9, beta_2=0.99, epsilon=1e-07, name="Nadam")

model.compile(
    optimizer=nadam, 
    loss="mean_absolute_error", 
    #loss=keras.losses.LogCosh(),
    metrics=["mean_absolute_error"]
)

es = EarlyStopping(monitor='mean_absolute_error', 
                   min_delta=1e-07, #min_delta=1e-04, 
                   patience=50, verbose=1)

history = model.fit(X_train, y_train, 
                    epochs=2000,
                    batch_size=X_train.shape[1] * 100,
                    callbacks=[TqdmCallback(verbose=0), es],
                    validation_data=[X_val, y_val],
                    verbose=0)

0epoch [00:00, ?epoch/s]

Epoch 742: early stopping


In [144]:
test_predicton = model.predict(X_test)
mae_error = mean_absolute_error(y_test, test_predicton)
print(mae_error)

67/67 [==============================] - 0s 2ms/step
3.7498239037533567


- Bukhsh partition (train, test, split_validation=0.5) = 3.7498
----
- Silva partition (train, test, split_validation=0.5) = 3.9524